In [1]:
#dependencies
#pip install polars
#pip install pandas

   ---------------------------------------- 0.0/802.4 kB ? eta -:--:--
   --------------------------------------- 802.4/802.4 kB 17.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/44.5 MB ? eta -:--:--
   ------------- -------------------------- 14.7/44.5 MB 75.7 MB/s eta 0:00:01
   ----------------------------- ---------- 32.8/44.5 MB 77.1 MB/s eta 0:00:01
   ---------------------------------------  44.3/44.5 MB 73.3 MB/s eta 0:00:01
   ---------------------------------------  44.3/44.5 MB 73.3 MB/s eta 0:00:01
   ---------------------------------------- 44.5/44.5 MB 49.8 MB/s eta 0:00:00

   ---------------------------------------- 0/2 [polars-runtime-32]
   ---------------------------------------- 0/2 [polars-runtime-32]
   ---------------------------------------- 0/2 [polars-runtime-32]
   ---------------------------------------- 0/2 [polars-runtime-32]
   ---------------------------------------- 0/2 [polars-runtime-32]
   ----------------------------------------

In [8]:
#Importing Libraries

import polars as pl
import pandas as pd
import glob
from pathlib import Path

# Reading the data

The first step is to read in the data. The folder path is defined in "Folder" and the files are read in using list comprehension. Each file that ends in ".csv" is appended to the dfs list. Additionally, to identify the source of each datapoint the name of the list has been appended to each entry

In [68]:
folder = Path(r"C:/Users/Tim/Documents/Coding Projects/test folder")

dfs = [(pl.read_csv(f).with_columns( pl.lit(f.stem).alias("file_name") )) for f in folder.glob("*.csv")]



In [19]:
folder = Path(r"C:/Users/Tim/Documents/Coding Projects/test folder")
list_of_dfs = []
for f in folder.glob("*.csv"):
    df = pd.read_csv(f)
    df[f.stem] = f.stem
    list_of_dfs.append(df)
    

In [33]:
for df in list_of_dfs:
    print(df.columns)

Index(['Name', 'Street', 'City', 'Zip', 'Country', 'Company',
       'clientlist2025'],
      dtype='object')
Index(['First Name', 'Last Name', 'Street', 'City', 'Zip', 'Country',
       'Company', 'contacts_A'],
      dtype='object')
Index(['First Name', 'Last Name', 'Street', 'City', 'Zip', 'Country',
       'Company', 'list_file_final'],
      dtype='object')
Index(['First Name', 'Last Name', 'Street', 'City', 'Zip', 'Country',
       'Company', 'random_contacts'],
      dtype='object')
Index(['Name', 'Street', 'City', 'Zip', 'Country', 'Company', 'TeamDirectory'], dtype='object')


In [69]:
dfs

[shape: (57, 7)
 ┌─────────────────┬────────────────┬────────────┬───────┬─────────┬───────────────┬────────────────┐
 │ Name            ┆ Street         ┆ City       ┆ Zip   ┆ Country ┆ Company       ┆ file_name      │
 │ ---             ┆ ---            ┆ ---        ┆ ---   ┆ ---     ┆ ---           ┆ ---            │
 │ str             ┆ str            ┆ str        ┆ str   ┆ str     ┆ str           ┆ str            │
 ╞═════════════════╪════════════════╪════════════╪═══════╪═════════╪═══════════════╪════════════════╡
 │ Jordan McDonald ┆ 5035 Maple St  ┆ Fairfield  ┆       ┆ USA     ┆ Innotech      ┆ clientlist2025 │
 │ Jordan Núñez    ┆ 6649 Pine Rd   ┆            ┆       ┆ Germany ┆ ČEZ Group     ┆ clientlist2025 │
 │ Zoë Johnson     ┆ 9651 Cedar Dr  ┆ Fairfield  ┆ 65953 ┆ Canada  ┆ Globex        ┆ clientlist2025 │
 │ Zoë Johnson     ┆ 9651 Cedar Dr  ┆ Fairfield  ┆ 65953 ┆ Canada  ┆ Globex        ┆ clientlist2025 │
 │ Leo Turner      ┆ 2836 Main St   ┆ Montréal   ┆ 80309 ┆ Germany

Question: If we want to apply conditions to one Dataframe but not others how would we do it? A simple solution lies in the structure of our data. Remember that our dataframes our in a list and that any value in that list can be called by dfs[x] and then you can do any operation you would normally do on any other dataframe. See below:

In [56]:
#Printing column names using for loop
for df in dfs:
   print( df.columns)

['Name', 'Street', 'City', 'Zip', 'Country', 'Company']
['First Name', 'Last Name', 'Street', 'City', 'Zip', 'Country', 'Company']
['First Name', 'Last Name', 'Street', 'City', 'Zip', 'Country', 'Company']
['First Name', 'Last Name', 'Street', 'City', 'Zip', 'Country', 'Company']
['Name', 'Street', 'City', 'Zip', 'Country', 'Company']


In [57]:
#Using conditional to identify structure

if "Name" in dfs[0].columns:
   print("YES")
else:
    print("No")


YES


# What is the use of this?

We now have a way to address the different column structures to standardize the dataframes. Remember, to union all dataframes must have the same column structure. Additionally, we can now build the cleaning procedure in such a way that it doesn't necessarily have to perform all the same steps on each individual dataframes if we end cleaning by looping or mapping. Let us test below adding the First Name and Last Name column we would need to standardize.

In [63]:
if "Name" in dfs[0].columns:
   df = dfs[0].with_columns([
    pl.col("Name").str.split(" ", inclusive=False).list.get(0).alias("First Name"),
    pl.col("Name").str.split(" ", inclusive=False).list.get(1).alias("Last Name")]).drop("Name")
else:
    print("No")

In [64]:
df

Street,City,Zip,Country,Company,First Name,Last Name
str,str,str,str,str,str,str
"""5035 Maple St""","""Fairfield""","""""","""USA""","""Innotech""","""Jordan""","""McDonald"""
"""6649 Pine Rd""","""""","""""","""Germany""","""ČEZ Group""","""Jordan""","""Núñez"""
"""9651 Cedar Dr""","""Fairfield""","""65953""","""Canada""","""Globex""","""Zoë""","""Johnson"""
"""9651 Cedar Dr""","""Fairfield""","""65953""","""Canada""","""Globex""","""Zoë""","""Johnson"""
"""2836 Main St""","""Montréal""","""80309""","""Germany""","""Innotech""","""Leo""","""Turner"""
…,…,…,…,…,…,…
"""7760 Oak Ave""","""Montréal""","""""","""USA""","""Umbrella Corp""","""Mike""","""McDonald"""
"""8839 Pine Rd""","""Greenville""","""73847""","""Canada""","""Wonka Inc""","""Jordan""","""Davis"""
"""5908 Pine Rd""","""Fairfield""","""28294""","""Canada""","""Umbrella Corp""","""Sasha""","""Young"""


In [78]:
#The Function

def fix_columns(list_of_dfs):
    #initialize list
    returned_list = []
    #loop through input list
    for df in list_of_dfs:
        #use our discovery
        if "Name" in df.columns:
            df = df.with_columns([
            pl.col("Name").str.split(" ", inclusive=False).list.get(0).alias('First Name'),
            pl.col("Name").str.split(" ", inclusive=False).list.get(1).alias('Last Name')]).drop("Name")
            #append to return list
            returned_list.append(df)
        else:
            #append dataframes that do not meet criteria to returned list
            returned_list.append(df)
            continue
    
    return returned_list

In [79]:
standard_columns = fix_columns(dfs)

# Union the Dataframes

Now that the columns are standardized, the next step is to combine the dataframes together to begin the cleaning process. 

In [82]:
#diagonal relaxed allows us to have the columns in a different order accross dataframes
combined_df = pl.union(standard_columns,  how="diagonal_relaxed")

In [83]:
combined_df

Street,City,Zip,Country,Company,file_name,First Name,Last Name
str,str,str,str,str,str,str,str
"""5035 Maple St""","""Fairfield""","""""","""USA""","""Innotech""","""clientlist2025""","""Jordan""","""McDonald"""
"""6649 Pine Rd""","""""","""""","""Germany""","""ČEZ Group""","""clientlist2025""","""Jordan""","""Núñez"""
"""9651 Cedar Dr""","""Fairfield""","""65953""","""Canada""","""Globex""","""clientlist2025""","""Zoë""","""Johnson"""
"""9651 Cedar Dr""","""Fairfield""","""65953""","""Canada""","""Globex""","""clientlist2025""","""Zoë""","""Johnson"""
"""2836 Main St""","""Montréal""","""80309""","""Germany""","""Innotech""","""clientlist2025""","""Leo""","""Turner"""
…,…,…,…,…,…,…,…
"""110 2nd St.""","""Greenville""","""70570""","""USA""","""Wonka Inc""","""TeamDirectory""","""Sasha""","""Lopez"""
"""110 2nd St.""","""Greenville""","""70570""","""USA""","""Wonka Inc""","""TeamDirectory""","""Sasha""","""Lopez"""
"""234 2nd St.""","""Riverton""","""""","""Canada""","""Soylent""","""TeamDirectory""","""Mike""","""Johnson"""


In [104]:
#checking file name assures us that the files merged correctly
combined_df['Country'].unique()

Country
str
"""Germany"""
"""USA"""
"""Canada"""


# Cleaning steps

This AI generated data has relatively few cleaning steps needed relative to a organic dataset. I will throw in a few steps that would probably be necessary for a more complex dataset.

In [147]:
pl.col("Country").str.len_chars()

<Expr ['col("Country").str.len_chars()'] at 0x2AD1EDC2150>

# Cleaning
The data is relatively clean containg blanks and duplicates. In order to simulate other common errors I will include steps that might be relevant to a more messy list. It also technically has the wrong format for international zip codes which I will factor in.

In [163]:
def clean_list(messy_list):
    step_1 = messy_list.unique(keep = "first")
    step_2 = step_1.filter(~pl.any_horizontal([pl.col("*") == ""]))

    ## note Canada would be along the lines of r"^[A-Za-z]\d[A-Za-z][ -]?\d[A-Za-z]\d$" but if we entered that it would blow up the data
    step_3 = step_2.filter(
        ((pl.col("Country") == "Germany") & (pl.col("Zip").str.contains(r"^\d{5}$")))|
        ((pl.col("Country") == "USA") & ((pl.col("Zip").str.contains(r"^\d{5}$")) | (pl.col("Zip").str.contains(r"^\d{5}[-]\d{4}"))))|
        ((pl.col("Country") == "Canada") & (pl.col("Zip").str.contains(r"^\d{5}$"))))
    final_output = step_3.filter(pl.col("Street").str.len_chars() > 4)
    return final_output
    

In [164]:
clean_list(combined_df)

Street,City,Zip,Country,Company,file_name,First Name,Last Name
str,str,str,str,str,str,str,str
"""1877 Elm St""","""Riverton""","""18058""","""Canada""","""Wonka Inc""","""contacts_A""","""Zoë""","""Davis"""
"""3834 Cedar Dr""","""Riverton""","""54725""","""Canada""","""Globex""","""clientlist2025""","""John""","""Chen"""
"""8928 Oak Ave""","""Lakeview""","""99524""","""USA""","""Wonka Inc""","""TeamDirectory""","""Sasha""","""Johnson"""
"""4356 Oak Ave""","""Springfield""","""53553""","""Germany""","""Globex""","""contacts_A""","""Bob""","""O'Connor Jr."""
"""555 Main St""","""München""","""46357""","""Germany""","""Globex""","""contacts_A""","""Ana María""","""Núñez"""
…,…,…,…,…,…,…,…
"""8753 2nd St.""","""Lakeview""","""66013""","""USA""","""ČEZ Group""","""list_file_final""","""Jiří""","""Johnson"""
"""9938 Oak Ave""","""Riverton""","""53965""","""USA""","""Umbrella Corp""","""list_file_final""","""sasha""","""Singh Sr."""
"""8662 Cedar Dr""","""München""","""65385""","""Canada""","""Stark Industries""","""clientlist2025""","""Jiří""","""Singh"""
